In [1]:
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
from pathlib import Path

import sys

import os



from astropy.nddata import CCDData
from astropy.visualization import hist
import ccdproc as ccdp
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

from matplotlib.colors import LogNorm
from PIPELINE.convenience_functions import show_image

from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table
import matplotlib.gridspec as gridspec

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import importlib



from astropy.stats import mad_std

from itertools import groupby

from PIL import Image as P

In [3]:
def jpeg_pixel_map(data,s1=-10,s2=10):
    d = data.astype('f')
    m = np.median(d)
    s = 1.49*np.median(np.abs(d-m))
    z1 = m-s1*s
    z2 = m+s1*s
    pixels = np.clip(255*(d-z1)/(z2-z1),0,255).astype("b")
    pixmap = P.frombytes("L",(pixels.shape[1],pixels.shape[0]),pixels.ravel().tostring())
    image = P.merge("L",[pixmap])
    
    return image

In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display,Image
import plotly.express as px
from skimage import io


def check_images(snight):
    # Create widgets
    directory = widgets.Dropdown(options=[snight.raw_data_dir,snight.processing_dir])
    images = widgets.Dropdown(options=os.listdir(directory.value))

    # Updates the image options based on directory value
    def update_images(*args):
        images.options = os.listdir(directory.value)

    # Tie the image options to directory value
    directory.observe(update_images, 'value')

    def show_images(fdir,file):

        fname = f'{fdir}/{file}'


        full_im_ccd = CCDData.read(fname,unit='adu')

        #full_im_pixel_map = jpeg_pixel_map(full_im_ccd.data,s1=10,s2=full_im_data.flatten().max())


        fig = px.imshow(full_im_ccd.data,color_continuous_scale='gray')

        display(fig)

    _ = interact(show_images,fdir=directory,file=images)

In [5]:
raw_goodman_path = "/Users/danakoeppe/PipelineProjects/Make_FITS_for_SAMOS/SAMI_reduction/GOODMAN_raw_data/"


SAMOS_raw_path = Path(raw_goodman_path)
                    #Path(\'./fake_raw_SAMOS_data/')

SAMOS_raw = ccdp.ImageFileCollection(SAMOS_raw_path)

SAMOS_raw.summary['file','obstype','exptime','rdnoise','gain']

file,obstype,exptime,rdnoise,gain
str26,str6,float64,float64,float64
psg_140319_210352_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_210620_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_210744_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_210907_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_211031_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_211154_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_211318_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_211442_fri.fits,FLAT,2.0,3.99,2.06
psg_140319_211605_fri.fits,FLAT,2.0,3.99,2.06


In [24]:
from PIPELINE import SAMOS_NIGHT as SN
importlib.reload(SN)

import logging
importlib.reload(logging)



log_filename = "SAMOS_DRP_GMOS_test.log"

if os.path.exists(log_filename):
    os.remove(log_filename)

log_format = '[%(asctime)s][%(levelname)8s]: %(message)s ' \
                     '[%(module)s.%(funcName)s:%(lineno)d]'
logging_level = logging.DEBUG

date_format = '%H:%M:%S'

formatter = logging.Formatter(fmt=log_format,
                              datefmt=date_format)

logging.basicConfig(filename=log_filename,
                    level=logging_level,
                    format=log_format,
                    datefmt=date_format)

file_handler = logging.FileHandler(filename=log_filename)
file_handler.setFormatter(fmt=formatter)
file_handler.setLevel(level=logging_level)

log = logging.getLogger(__name__)

log.info('hi')

In [25]:
log.info("Testing main log updates.")


In [26]:
from PIPELINE import SAMOS_NIGHT as SN
importlib.reload(SN)


#log_filename = "SAMOS_DRP_GMOS_test.log"

SNight = SN.SAMOSNight(raw_data_dir=raw_goodman_path,
                       obsid="fake_samos_test",
                       proc_dir="processed",
                       LOG_FILENAME=log_filename,
                       ignore_bias=False,
                       ignore_flats=False)

SNight()
log.name

PIPELINE.SAMOS_NIGHT
the pipeline is sorting your buckets
True
{'obsid': 'fake_samos_test', 'data_path': '/Users/danakoeppe/PipelineProjects/Make_FITS_for_SAMOS/SAMI_reduction/GOODMAN_raw_data/', 'gain': 2.06, 'rdnoise': 3.99, 'ccdsum': '1 1', 'empty_bucket': True, 'bias_buckets': None, 'flat_buckets': None, 'comp_buckets': None, 'targ_buckets': None, 'spec_buckets': None, 'slit_buckets': None, 'fits_slit_locs': None, 'sunset_time': None, 'sunrise_time': None, 'evening_twilight': None, 'morning_twilight': None, 'header_keys': ['naxis', 'date', 'slit', 'date-obs', 'obstype', 'object', 'exptime', 'obsra', 'obsdec', 'grating', 'cam_targ', 'grt_targ', 'filter', 'filter2', 'gain', 'rdnoise', 'ccdsum', 'wavmode']}
added flat bucket                          file  naxis        date                 slit  \
0  psg_140319_210352_fri.fits      2  2014-03-19  Ori0B1-field2-0000R   
1  psg_140319_210620_fri.fits      2  2014-03-19  Ori0B1-field2-0000R   
2  psg_140319_210744_fri.fits      2  2014-03

'__main__'

In [27]:
import PIPELINE.ImageProcessor as ImProc
import PIPELINE.SAMOS_mods as Smods
importlib.reload(Smods)
importlib.reload(ImProc)


SN_ccd = ImProc.ImageProcessor(SNight)
SN_ccd()



INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


/Users/danakoeppe/PipelineProjects/Make_FITS_for_SAMOS/SAMI_reduction/GOODMAN_raw_data/
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
processed/master_flat_400 m2_GG455_01-2-0000.fits processed/master_flat_400 m2_GG455_01-2-0000.fits
(1896, 4081) (1896, 4081)


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
science_image shape: (1896, 4142), master_bias shape: (1896, 4142)
psg_140320_010553_cri.fits master_bias_1x1_R3d99_G2d06.fits
master bias shape: (1896, 4142)
 current ccd shape: (1896, 4142)


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
processed/master_flat_400 m2_GG455_01-2-0000.fits processed/master_flat_400 m2_GG455_01-2-0000.fits
(1896, 4081) (1896, 4081)
Empty DataFrame
Columns: [x_edge_left, x_edge_right, y_mid]
Index: [] Index([], dtype='object')


In [16]:
SN_ccd.processing_dir

'processed'

In [17]:
fnames = []


for f in os.listdir(SN_ccd.processing_dir):
    
    if "ftz" in f:
        fnames.append(SN_ccd.processing_dir+"/"+f)
        
fnames

['processed/ftzpsg_140320_010018_cri.fits',
 'processed/ftzpsg_140320_003057_ori.fits',
 'processed/ftzpsg_140320_005731_cri.fits',
 'processed/ftzpsg_140320_001436_ori.fits',
 'processed/ftzpsg_140319_235651_ori.fits',
 'processed/ftzpsg_140320_005607_cri.fits',
 'processed/ftzpsg_140320_010429_cri.fits',
 'processed/ftzpsg_140320_010142_cri.fits',
 'processed/ftzpsg_140320_005443_cri.fits',
 'processed/ftzpsg_140320_010553_cri.fits',
 'processed/ftzpsg_140320_005007_cri.fits',
 'processed/ftzpsg_140320_010305_cri.fits',
 'processed/ftzpsg_140320_005854_cri.fits',
 'processed/ftzpsg_140320_005319_cri.fits']

In [12]:
import PIPELINE.SlitBuckets as Slits
import PIPELINE.slit_tracing_mods as stm
importlib.reload(Slits)
importlib.reload(stm)



SN_slits = Slits.SlitBuckets(SN_ccd)
SN_slits()


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [15]:

SN_slits.log




<Logger PIPELINE.SlitBuckets (WARNING)>

In [14]:
import PIPELINE.DoWavecal as DoWv
importlib.reload(DoWv)
import PIPELINE.Spectroscopy.wavelength
importlib.reload(PIPELINE.Spectroscopy.wavelength)
import PIPELINE.Spectroscopy.wcs
importlib.reload(PIPELINE.Spectroscopy.wcs)


SN_wv = DoWv.WaveCalBuckets(SN_slits)

SN_wv()

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


reference lamp location: /Users/danakoeppe/PipelineProjects/SAMOS_DRP/SAMOS_Draft/comp_refs
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
True
TESTING


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


s1_ftzmt_140319_235651_ori.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


KeyError: "Keyword 'GRP_FUNC' not found."

In [30]:
from astropy.modeling import models
from PIPELINE.SAMOS_mods import read_fits


ccd = read_fits("processed/Ori0B1-field2-0000R/s1_ftzmt_140319_235651_ori.fits")

model = models.Chebyshev1D(degree=3)
print(type(model))
for i in range(int(3) + 1):
    model.__getattr__('c{:d}'.format(i)).value = ccd.header[
        'GRP_C{:03d}'.format(i)]
    wavelength_and_intensity = [
    model(range(ccd.header['GRP_NPIX'])), ccd.data]

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
<class 'astropy.modeling.polynomial.Chebyshev1D'>
Name: Chebyshev1D
N_inputs: 1
N_outputs: 1
Fittable parameters: <property object at 0x7fe3520a8130>


KeyError: "Keyword 'GRP_C000' not found."

In [47]:
SNight.log.info

<bound method Logger.info of <Logger PIPELINE.SAMOS_NIGHT (WARNING)>>

In [38]:
SN_slits.work_dir

'/Users/danakoeppe/PipelineProjects/SAMOS_DRP/SAMOS_Draft'